In [1]:
#import the libraries needed
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from scipy import stats
import datetime

In [2]:
#Load the train values from the train_values.csv file
df = pd.read_csv("train_values.csv")

#Load the label values from the label.csv file for each building id
label = pd.read_csv("train_labels.csv")

In [3]:
#get first 2 of train_values
df.head(2)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [4]:
#get first 2 train_labels
label.head(10)

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional
5,9944,functional
6,19816,non functional
7,54551,non functional
8,53934,non functional
9,46144,functional


In [5]:
label.head(10)

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional
5,9944,functional
6,19816,non functional
7,54551,non functional
8,53934,non functional
9,46144,functional


In [6]:
label_encoder = preprocessing.LabelEncoder()
label['status_group'] = label_encoder.fit_transform(label['status_group'])
# functional 0, non functional - 2 , functional need repair - 1

In [7]:
label.head(8)

,id,status_group
0,69572,0
1,8776,0
2,34310,0
3,67743,2
4,19728,0
5,9944,0
6,19816,2
7,54551,2


In [8]:
#find whether dataset contains any missing values in any of the columns
df.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [9]:
# Find columns with Null values
nullcols = []
for col in df.columns:
    nbnull = (df[col].isnull()*1).sum()
    if ( nbnull > 0 ):
        tp = type(df[df[col].notnull()][col].iat[0]) # type of first non null value
        nullcols.append([col, tp])
        print(col, nbnull, tp)

funder 3635 <class 'str'>
installer 3655 <class 'str'>
subvillage 371 <class 'str'>
public_meeting 3334 <class 'bool'>
scheme_management 3877 <class 'str'>
scheme_name 28166 <class 'str'>
permit 3056 <class 'bool'>


In [10]:
for col, tp in nullcols:
    if (tp == str):
        df.loc[df[col].isnull(), col] = 'MISSING'
        
for col, tp in nullcols:
    if (tp == bool):
        df.loc[df[col]==True, col] = 'TRUE'
        df.loc[df[col]==False, col] = 'FALSE'
        df.loc[df[col].isnull(), col] = 'MISSING'

In [11]:
#Drop columns not necessary for the model
drop_list = [
             'recorded_by',
             'waterpoint_type_group',
             'extraction_type_group',
             'recorded_by',
             'quality_group',
             'quantity_group',
             'id',
             'source',
             'source_class'
            ]
df = df.drop(drop_list,axis=1)

In [12]:
df.isnull().sum()

amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
scheme_management        0
scheme_name              0
permit                   0
construction_year        0
extraction_type          0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quantity                 0
source_type              0
waterpoint_type          0
dtype: int64

In [13]:
dates=[]
# before filling the null keep track of them
df['construction_year_missing'] = (df['construction_year']==0)*1
dates.append( 'construction_year_missing' ) # list of dates related fields

# to fill missing dates, can use : mean, median or oldest
mean_year = df[df['construction_year']>0]['construction_year'].mean()

df.loc[df['construction_year']==0, 'construction_year'] = int(mean_year)

In [14]:
df['date_recorded'] = pd.to_datetime(df['date_recorded'], errors='coerce')
now = pd.to_datetime('now')
df['time_for_record'] = ((now - df['date_recorded']).dt.total_seconds() / (60*60*24*365.25)).round(0)

In [15]:
# bound of min/max latitude/longitude/height for Tanzania
bound_df = df[(df['latitude']<-0.5)&(df['longitude']>25)&(df['gps_height']>0)]

# mean of geographical data in each bucket
mean_geo_df = bound_df.groupby(['basin',])['latitude','longitude','gps_height'].mean()

assert(mean_geo_df.shape[0] == len(df['basin'].unique()))

mean_geo_df

,latitude,longitude,gps_height
basin,,,
Internal,-4.137060,35.471586,1474.550470
Lake Nyasa,-10.021460,34.772641,1524.654237
Lake Rukwa,-7.489675,31.500196,1415.137352
Lake Tanganyika,-4.938659,30.398291,1270.063242
Lake Victoria,-1.868478,34.004127,1329.265008
Pangani,-3.849383,37.546848,1102.257742
Rufiji,-8.483380,35.852000,1087.780631
Ruvuma / Southern Coast,-10.547087,38.156886,455.058376
Wami / Ruvu,-6.712766,38.170352,308.914208


In [16]:
geos=[]
# Before overwriting keep track of suspect rows with new binary columns
df['gps_height_bad'] = (df['gps_height']<=0)*1
geos.append('gps_height_bad')
df['longitude_bad'] = (df['longitude']<25)*1
geos.append('longitude_bad')
df['latitude_bad'] = (df['latitude']>-0.5)*1
geos.append('latitude_bad')

# Exemple of query via index=basin : mean_geo_df.at['Lake Victoria','latitude']
df.loc[df['gps_height']<=0, 'gps_height'] = df['basin'].apply(lambda x : mean_geo_df.at[x,'gps_height'])
df.loc[df['longitude']<25, 'longitude'] = df['basin'].apply(lambda x : mean_geo_df.at[x,'longitude'])
df.loc[df['latitude']>-0.5, 'latitude'] = df['basin'].apply(lambda x : mean_geo_df.at[x,'latitude'])

In [17]:
columns_to_delete = ["date_recorded"]
df.drop(labels=columns_to_delete, axis="columns", inplace=True)

In [18]:
#df['target'] = label['status_group']

In [19]:
#plt.figure(figsize=(12,12))
#sns.scatterplot(x='longitude', y='latitude',data=df,hue='target',style='target')
#plt.ylim(-12, 0)
#plt.xlim(27, 42)
#plt.show()

In [20]:
#sns.set(style="whitegrid")
#sns.barplot(x="target", y="population", data=df)

In [21]:
value_replacement = {'commercial':'other', 'parastatal':'other', 'unknown':'other'}
df['management_group'].replace(value_replacement, inplace=True)

In [22]:
value_replacement = {'dam':'other', 'rainwater harvesting':'other'}
df['source_type'].replace(value_replacement, inplace=True)

In [23]:
value_replacement = {'salty':'other',
                     'milky':'other', 
                     'unknown':'other', 
                     'fluoride':'other', 
                     'coloured':'other',
                     'salty abandoned':'other', 
                     'fluoride abandoned':'other'
                    }
df['water_quality'].replace(value_replacement,inplace=True)

In [24]:
df.dtypes

amount_tsh                   float64
funder                        object
gps_height                   float64
installer                     object
longitude                    float64
latitude                     float64
wpt_name                      object
num_private                    int64
basin                         object
subvillage                    object
region                        object
region_code                    int64
district_code                  int64
lga                           object
ward                          object
population                     int64
public_meeting                object
scheme_management             object
scheme_name                   object
permit                        object
construction_year              int64
extraction_type               object
extraction_type_class         object
management                    object
management_group              object
payment                       object
payment_type                  object
w

In [25]:
#Object data encode with numerical data
df_objects = df.select_dtypes(include=['object'])
label_encoder = preprocessing.LabelEncoder()
for col in df_objects.columns:
    df[col] = label_encoder.fit_transform(df[col])

In [26]:
#convert dataframe into a numpy array
X = df.values
Y = label.values
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [27]:
Y=Y[:,1] #to remove id column

In [28]:
#X_train = X_train
#Y_train = Y_train[:,1]
#X_test = X_test
#Y_test = Y_test[:,1]

In [29]:
randomForestModel = RandomForestClassifier(n_estimators=80)

In [30]:
def getScore(model,X_train,Y_train,X_test,Y_test):
    model.fit(X_train,Y_train)
    return model.score(X_test,Y_test)

In [31]:
kfold = StratifiedKFold(n_splits=10)
kfold

StratifiedKFold(n_splits=10, random_state=None, shuffle=False)

In [32]:
RF_score=[]

for train_index,test_index in kfold.split(X,Y):
    X_train,X_test,Y_train,Y_test = X[train_index],X[test_index],Y[train_index],Y[test_index]
    RF_score.append(getScore(randomForestModel,X_train,Y_train,X_test,Y_test))

In [33]:
RF_score

[0.8156875946810301,
 0.8173708129944454,
 0.8170341693317623,
 0.8040733883184649,
 0.8217171717171717,
 0.802020202020202,
 0.8146464646464646,
 0.8046809227142616,
 0.8006398383566257,
 0.8152576625126305]

In [34]:
avg_rf_classification_score = sum(RF_score)/len(RF_score)
print(avg_rf_classification_score)

0.8113128227293058


In [35]:
#Load the test values from the Test_set_values.csv file for predictions
test_for_pred = pd.read_csv("Test_set_values.csv")

In [36]:
from sklearn.metrics import accuracy_score
predictions = randomForestModel.predict(X_train)
accuracy_score(Y_train,predictions)

0.9999251805020388

In [37]:
def preprocesing(df):
    #values = {'permit': 1,'scheme_management':'Other'}
    #df.fillna(value=values,inplace = True)
    # Find columns with Null values
    nullcols = []
    for col in df.columns:
        nbnull = (df[col].isnull()*1).sum()
        if ( nbnull > 0 ):
            tp = type(df[df[col].notnull()][col].iat[0]) # type of first non null value
            nullcols.append([col, tp])
    
    for col, tp in nullcols:
        if (tp == str):
            df.loc[df[col].isnull(), col] = 'MISSING'
        
    for col, tp in nullcols:
        if (tp == bool):
            df.loc[df[col]==True, col] = 'TRUE'
            df.loc[df[col]==False, col] = 'FALSE'
            df.loc[df[col].isnull(), col] = 'MISSING'
    
    #Drop columns not necessary for the model
    drop_list = [
                 'recorded_by',
                 'waterpoint_type_group',
                 'extraction_type_group',
                 'recorded_by',
                 'quality_group',
                 'quantity_group',
                 'id',
                 'source',
                 'source_class'
                ]
    df = df.drop(drop_list,axis=1)
    
    dates=[]
    # before filling the null keep track of them
    df['construction_year_missing'] = (df['construction_year']==0)*1
    dates.append( 'construction_year_missing' ) # list of dates related fields

    # to fill missing dates, can use : mean, median or oldest
    mean_year = df[df['construction_year']>0]['construction_year'].mean()

    df.loc[df['construction_year']==0, 'construction_year'] = int(mean_year)
    
    
    

    df['date_recorded'] = pd.to_datetime(df['date_recorded'], errors='coerce')
    now = pd.to_datetime('now')
    df['time_for_record'] = ((now - df['date_recorded']).dt.total_seconds() / (60*60*24*365.25)).round(0)
    
    #df['time_to_construction_year'] = datetime.datetime.now().year - df['construction_year']
    
    #df['time_to_construction_year'] = np.where(df['time_to_construction_year'] == 2019, df['time_for_record'], df['time_to_construction_year'])
    
    
    # bound of min/max latitude/longitude/height for Tanzania
    bound_df = df[(df['latitude']<-0.5)&(df['longitude']>25)&(df['gps_height']>0)]

    # mean of geographical data in each bucket
    mean_geo_df = bound_df.groupby(['basin',])['latitude','longitude','gps_height'].mean()

    assert(mean_geo_df.shape[0] == len(df['basin'].unique()))
    
    geos=[]
    # Before overwriting keep track of suspect rows with new binary columns
    df['gps_height_bad'] = (df['gps_height']<=0)*1
    geos.append('gps_height_bad')
    df['longitude_bad'] = (df['longitude']<25)*1
    geos.append('longitude_bad')
    df['latitude_bad'] = (df['latitude']>-0.5)*1
    geos.append('latitude_bad')

    # Exemple of query via index=basin : mean_geo_df.at['Lake Victoria','latitude']
    df.loc[df['gps_height']<=0, 'gps_height'] = df['basin'].apply(lambda x : mean_geo_df.at[x,'gps_height'])
    df.loc[df['longitude']<25, 'longitude'] = df['basin'].apply(lambda x : mean_geo_df.at[x,'longitude'])
    df.loc[df['latitude']>-0.5, 'latitude'] = df['basin'].apply(lambda x : mean_geo_df.at[x,'latitude'])
    
    
    columns_to_delete = ["date_recorded"]
    df.drop(labels=columns_to_delete, axis="columns", inplace=True)
    
    value_replacement = {'commercial':'other', 'parastatal':'other', 'unknown':'other'}
    df['management_group'].replace(value_replacement, inplace=True)
    
    new_value_replacement = {'salty':'other',
                     'milky':'other', 
                     'unknown':'other', 
                     'fluoride':'other', 
                     'coloured':'other',
                     'salty abandoned':'other', 
                     'fluoride abandoned':'other'
                    }
    df['water_quality'].replace(new_value_replacement,inplace=True)
    
    #Object data encode with numerical data
    df_objects = df.select_dtypes(include=['object'])
    label_encoder = preprocessing.LabelEncoder()
    for col in df_objects.columns:
        df[col] = label_encoder.fit_transform(df[col])
        
    #convert dataframe into a numpy array
    X = df.values
    
    return randomForestModel.predict(X)

In [38]:
final_pred = preprocesing(test_for_pred)

In [39]:
# replace the old ones in the list with the new values 
result = pd.Series(final_pred).replace(to_replace = [0, 2, 1], value = ['functional', 'non functional', 'functional needs repair']) 

In [40]:
evaluation = pd.DataFrame({'id':test_for_pred.id,"status_group": result.values[:]})

In [41]:
evaluation.to_csv("Submission_07.csv",index=False)

In [42]:
label.head(5)

,id,status_group
0,69572,0
1,8776,0
2,34310,0
3,67743,2
4,19728,0


In [43]:
#for col in df.columns: # Check frequency of most common values
#    cs = df[col].value_counts(normalize=True, sort=True, ascending=False)
#    for k in cs.keys()[0:5]:
#        print( col, k, int(cs[k]*1000)/10. )

In [44]:
df.isnull().sum()

amount_tsh                   0
funder                       0
gps_height                   0
installer                    0
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                   0
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting               0
scheme_management            0
scheme_name                  0
permit                       0
construction_year            0
extraction_type              0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quantity                     0
source_type                  0
waterpoint_type              0
construction_year_missing    0
time_for